In [11]:
#投票分類
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

#moonセット利用
X,y=make_moons(n_samples=500,noise=0.3,random_state=42)    ##データの量が多いとSVCがアンサンブルを超す？
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC(probability=True)

voting_clf= VotingClassifier(
    estimators=[("lr",log_clf),("rf",rnd_clf),("svc",svm_clf)],
    voting="soft")
voting_clf.fit(X_train,y_train)

#正解率測定
from sklearn.metrics import accuracy_score
for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))
    
    
    

LogisticRegression 0.864
RandomForestClassifier 0.888
SVC 0.896
VotingClassifier 0.912


In [12]:
#バギング　ペースティング
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X,y=make_moons(n_samples=500,noise=0.3,random_state=42)    ##データの量が多いとSVCがアンサンブルを超す？
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

#バギング　OOB検証
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,bootstrap=True,n_jobs=-1,oob_score=True)
bag_clf.fit(X_train,y_train)
bag_clf.oob_score_
from sklearn.metrics import accuracy_score
y_pred=bag_clf.predict(X_test)
print(accuracy_score(y_pred,y_test))

bag_clf.oob_decision_function_

#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier

rnd_clf=RandomForestClassifier(n_estimators=500, max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)

y_pred_rf=rnd_clf.predict(X_test)

bag_clf=BaggingClassifier(
    DecisionTreeClassifier(splitter="random",max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0,bootstrap=True,n_jobs=-1)

0.904


In [13]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X,y=make_moons(n_samples=500,noise=0.3,random_state=42)    ##データの量が多いとSVCがアンサンブルの精度を超す？
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

#AdaBoost分類器
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200, algorithm="SAMME.R",learning_rate=0.5)
ada_clf.fit(X_train,y_train)

from sklearn.metrics import accuracy_score
y_pred=ada_clf.predict(X_test)
accuracy_score(y_pred,y_test)



0.896

In [14]:
##勾配ブースティング木
import numpy as np
np.random.seed(42)
X=np.random.rand(100,1)-0.5  ##〇以上1未満の乱数
y=3*X[:,0]**2+0.05*np.random.randn(100)
from sklearn.tree import DecisionTreeRegressor
tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

y2=y-tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)
y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

X_new = np.array([[0.8]])
y_pred=sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2,tree_reg3))

print(y_pred)

#勾配ブースティング木　sklearn使用
from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2,n_estimators=3,learning_rate=1.0)
gbrt.fit(X,y)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train,X_val,y_train,y_val=train_test_split(X,y)

gbrt=GradientBoostingRegressor(max_depth=2,n_estimators=120)
gbrt.fit(X_train,y_train)

errors=[mean_squared_error(y_val,y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators=np.argmin(errors)+1

gbrt_best=GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train,y_train)
gbrt_best.predict(X_new)

gbrt=GradientBoostingRegressor(max_depth=2,warm_start=True)

min_val_error=float("inf")
error_going_up=0
for n_estimators in range(1,120):
    gbrt.n_estimators=n_estimators
    gbrt.fit(X_train,y_train)
    y_pred=gbrt.predict(X_val)
    val_error=mean_squared_error(y_val,y_pred)
    if val_error < min_val_error:
        min_val_error=val_error
        error_going_up=0
    else:
        error_going_up+=1
        if error_going_up==5:
            break

            




[0.75026781]


In [10]:
#XGBoost
import xgboost

xgb_reg=xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred=xgb_reg.predict(X_val)

xgb_reg.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=2)
y_pred=xgb_reg.predict(X_val)

%timeit xgboost.XGBRegressor().fit(X_train,y_train) 
%timeit GradientBoostingRegressor().fit(X_train,y_train)

[0]	validation_0-rmse:0.22055
[1]	validation_0-rmse:0.16547
[2]	validation_0-rmse:0.12243
[3]	validation_0-rmse:0.10044
[4]	validation_0-rmse:0.08467
[5]	validation_0-rmse:0.07344
[6]	validation_0-rmse:0.06728
[7]	validation_0-rmse:0.06383
[8]	validation_0-rmse:0.06125
[9]	validation_0-rmse:0.05959
[10]	validation_0-rmse:0.05902
[11]	validation_0-rmse:0.05852
[12]	validation_0-rmse:0.05844
[13]	validation_0-rmse:0.05801
[14]	validation_0-rmse:0.05747
[15]	validation_0-rmse:0.05772
[16]	validation_0-rmse:0.05778
110 ms ± 444 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
48.2 ms ± 622 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
random_forest_clf=RandomForestClassifier(n_estimators=10,random_state=42) ##木のパラメータ指定　 ##mnist データサイズ指定　忘れないように
for estimator in estimators:   ###for を使って見やすく簡潔に
    print([estimator.score(X_val,y_val) for estimator in estimators])   ##属性指定　.scoreなどを用いて、accuracy_scoreではなくて